### Create one big table

In [0]:
stg_posts_df = spark.table("silver.stg_posts")

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

def posts_top_tags(stg_posts_df: DataFrame) -> DataFrame:
    return (
        stg_posts_df
        .withColumn("tag_exploded", F.explode("TagsArray"))
        .groupBy("tag_exploded").agg(F.approx_count_distinct("PostId").alias("tags_count"))
        .orderBy(F.col("tags_count").desc())
    )

marts_top_tags_df = posts_top_tags(stg_posts_df)
marts_top_tags_df.write.mode("overwrite").saveAsTable("gold.marts_top_tags")

In [0]:
display(spark.table("gold.marts_top_tags").limit(5))